In [1]:
"""OpenRouter LLM client initialization and configuration."""

import os

from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

load_dotenv()

model = "openai/gpt-5-mini"
temperature = 0.0


class Schema(BaseModel):
    answer: str = Field(description="The answer to the question")
    reason: str = Field(description="The reason for the answer")


llm = ChatOpenAI(
    model=model,
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url="https://openrouter.ai/api/v1",
    temperature=temperature,
    reasoning_effort="low",  # Changed from "minimal" - minimal doesn't work with web_search
    extra_body={"plugins": [{"id": "web"}]},
).with_structured_output(Schema)

In [2]:
from langchain_playground.llm.multimodal import MediaMessage

message = MediaMessage("input.png", description="Describe the weather in this image")
message

2025-11-25 16:38:20,295 - WARNING - USER_AGENT environment variable not set, consider setting it to identify your requests.


MediaMessage(content=[{'type': 'image', 'id': 'lc_a0cc1d77-4531-4ad9-b730-6f7481b435c9', 'base64': '...', 'mime_type': 'image/png'}, {'type': 'text', 'text': 'Describe the weather in this image', 'id': 'lc_c0bc4bfa-97fe-4dfa-9a8e-863ebaa840f0'}], additional_kwargs={}, response_metadata={})

In [3]:
# MediaMessage creates the message with content blocks automatically
messages = [message]

In [4]:
ai_msg = llm.invoke(messages)
ai_msg

2025-11-25 16:38:22,090 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Schema(answer='I can’t determine the weather from that image — it’s a screenshot of a code snippet (a JSON/Python-like object showing a message with text and an image URL), not a photograph of an outdoor scene. There are no visible sky, clouds, sun, precipitation, or other weather cues to describe.', reason='The image contains a code block listing a ‘message’ object and an example image URL, so it does not show any real-world environment or weather features. If you meant to upload a photo of the sky/outdoors, please upload that image and I’ll describe the weather.')

In [5]:
# With structured output, response is already a Pydantic model
response: Schema = llm.invoke("What is the price of GOOGL today")
print(f"Answer: {response.answer}")
print(f"Reason: {response.reason}")
response

2025-11-25 16:38:26,991 - INFO - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"


Answer: GOOGL (Alphabet Inc.) — $318.58 per share (last trade).
Reason: Price data from market feed (latest trade time: Tuesday, November 25, 2025 at 01:15:00 UTC). Source: finance lookup. citeturn0finance0


Schema(answer='GOOGL (Alphabet Inc.) — $318.58 per share (last trade).', reason='Price data from market feed (latest trade time: Tuesday, November 25, 2025 at 01:15:00 UTC). Source: finance lookup. \ue200cite\ue202turn0finance0\ue201')